In [1]:
import midi

In [155]:
b867 = midi.read_midifile('music/chpn-p10.mid')

In [13]:
b867

midi.Pattern(format=1, resolution=480, tracks=\
[midi.Track(\
  [midi.TrackNameEvent(tick=0, text='Das wohltemperierte Klavier I - Praeludium und Fuge 2 in c-Moll BWV 847', data=[68, 97, 115, 32, 119, 111, 104, 108, 116, 101, 109, 112, 101, 114, 105, 101, 114, 116, 101, 32, 75, 108, 97, 118, 105, 101, 114, 32, 73, 32, 45, 32, 80, 114, 97, 101, 108, 117, 100, 105, 117, 109, 32, 117, 110, 100, 32, 70, 117, 103, 101, 32, 50, 32, 105, 110, 32, 99, 45, 77, 111, 108, 108, 32, 66, 87, 86, 32, 56, 52, 55]),
   midi.CopyrightMetaEvent(tick=0, text='Copyright \xa9 1997 von Bernd Kr\xfcger.', data=[67, 111, 112, 121, 114, 105, 103, 104, 116, 32, 169, 32, 49, 57, 57, 55, 32, 118, 111, 110, 32, 66, 101, 114, 110, 100, 32, 75, 114, 252, 103, 101, 114, 46]),
   midi.TextMetaEvent(tick=0, text='Johann Sebastian Bach', data=[74, 111, 104, 97, 110, 110, 32, 83, 101, 98, 97, 115, 116, 105, 97, 110, 32, 66, 97, 99, 104]),
   midi.TextMetaEvent(tick=0, text='Fertiggestellt am 5.11.97\n', data=[70, 101, 114

In [2]:
from midi_to_statematrix import *

SM_deb = midiToNoteStateMatrix('music/debussy_cc_1.mid')

In [216]:
len(SM_bach)

3219

In [165]:
np.shape(SM_chp)

(1208, 78, 2)

In [132]:
np.shape(SM + SM2 + SM3)

(3219, 78, 2)

In [133]:
SM_bach = SM + SM2 + SM3

In [3]:
import numpy as np

In [4]:
from data import *

In [24]:
IF = noteStateMatrixToInputForm(SM)

In [15]:
batch = SM[100:100+16*8]

In [19]:
np.ceil(len(SM) / float(16*8))

9.0

In [25]:
len(SM)

1121

In [28]:
len(SM[1120:1128])

1

In [40]:
8*16*8

1024

In [6]:
def get_measure_groups(statematrix):
    batch_size = 16*1
    measures = []

    n_ticks = len(statematrix)
    
    for i in range(int(np.floor(n_ticks / float(batch_size)))):
        measures.append(statematrix[i*batch_size:(i+1)*batch_size])
    
#     # Pad the last group of 8-measures to make it the same length as the previous
#     if len(measures[-1]) < batch_size:
#         measures[-1] = 
        
    return measures

In [7]:
from statsmodels.nonparametric.kernel_density import KDEMultivariate

def get_KDE(generated_list):
    sm_tot = midiToNoteStateMatrix(generated_list[0])

    for f in generated_list[1:]:
        sm_tot += midiToNoteStateMatrix(f)
        
    mgs = get_measure_groups(sm_tot)

    print np.shape(mgs)
    
    KDE = KDEMultivariate(data=mgs*2, var_type='u'*78*16*2)
    
    return KDE

In [8]:
def get_ISL(dens, test_list):
    sm_test = midiToNoteStateMatrix(test_list[0])
    
    for f in test_list:
        sm_test += midiToNoteStateMatrix(f)
        
    test_mgs = get_measure_groups(sm_test)

    ISLs = []
    
    for measure in test_mgs:
        test_measure_flat = np.reshape(measure, 78*16*2)
        measure_ISL = np.log(dens.pdf(test_measure_flat))
        ISLs.append(measure_ISL)
        
    return ISLs

In [13]:
import sys, os

gdir = '../../../output_curriculum/output_curriculum_500iter/output_full/'

generated_list = [gdir + f for f in os.listdir(gdir) if 'composition_curric_9500_' in f]

dens = get_KDE(generated_list)

(1993, 16, 78, 2)


In [14]:
test_list = ['music/' + f for f in os.listdir('music/') if 'bach' in f and 'zip' not in f]

bach_ISL = get_ISL(dens, test_list)

In [15]:
non_nan = [f for f in bach_ISL if not np.isnan(f)]

In [16]:
np.mean(non_nan)

-377.79918583084924

In [214]:
mgs = get_measure_groups(SM_bach)
mgs_deb = get_measure_groups(SM_deb)

In [218]:
np.shape(mgs)

(201, 16, 78, 2)

In [217]:
len(mgs)

201

In [177]:
np.shape(mgs_deb)

(80, 16, 78, 2)

In [68]:
16*78*2

2496

In [142]:
len(mgs*20)

4020

In [159]:
np.shape(SM_chp)

(1, 78, 2)

In [30]:
np.array(IF).shape

(1105, 78, 80)

In [33]:
np.array(IF[0]).shape

(78, 80)

In [7]:
np.array(SM).shape

(1121, 78, 2)

In [12]:
from scipy.stats import gaussian_kde

In [19]:
sm_np = np.array(SM)

In [14]:
?gaussian_kde

In [54]:
from statsmodels.nonparametric.kernel_density import KDEMultivariate

In [47]:
SM2 = [[m[0] for m in x] for x in SM]

In [55]:
np.shape(SM2)

(1105, 78)

In [65]:
a = [SM2]*100

In [80]:
SM2_arr = np.array(SM2)

In [105]:
SM2_arr.shape

(1105, 78)

In [106]:
a = [SM2_arr] * 10

In [120]:
np.shape(mgs_new)

(62, 16, 78, 2)

In [215]:
dens = KDEMultivariate(data=mgs*20, var_type='u'*78*16*2, bw='cv_ls')

KeyboardInterrupt: 

In [213]:
mgs_flat = np.reshape(mgs_deb[16], 78*16*2)
np.log(dens.pdf(mgs_flat))

-351.28725271181963

In [5]:
import os

In [101]:
a = [dens.pdf(x) for x in SM2_arr]

In [6]:
import cPickle as pickle
import gzip
import numpy
from midi_to_statematrix import *

import multi_training
import model

def gen_adaptive(m, pcs, times, keep_thoughts=False, name="final"):
    xIpt, xOpt = map(lambda x: numpy.array(x, dtype='int8'), multi_training.getPieceSegment(pcs))
    all_outputs = [xOpt[0]]
    if keep_thoughts:
        all_thoughts = []
    m.start_slow_walk(xIpt[0])
    cons = 1
    for time in range(multi_training.batch_len * times):
        resdata = m.slow_walk_fun(cons)
        nnotes = numpy.sum(resdata[-1][:, 0])
        if nnotes < 2:
            if cons > 1:
                cons = 1
            cons -= 0.02
        else:
            cons += (1 - cons) * 0.3
        all_outputs.append(resdata[-1])
        if keep_thoughts:
            all_thoughts.append(resdata)
    noteStateMatrixToMidi(numpy.array(all_outputs), name)
    if keep_thoughts:
        pickle.dump(all_thoughts, open('output/' + name + '.p', 'wb'))


def fetch_train_thoughts(m, pcs, batches, name="trainthoughts"):
    all_thoughts = []
    for i in range(batches):
        ipt, opt = multi_training.getPieceBatch(pcs)
        thoughts = m.update_thought_fun(ipt, opt)
        all_thoughts.append((ipt, opt, thoughts))
    pickle.dump(all_thoughts, open('output/' + name + '.p', 'wb'))

pcs = multi_training.loadPieces("music")

m = model.Model([300, 300], [100, 50], dropout=0.5)

WARNING (theano.gof.cmodule): WARNING: your Theano flags `gcc.cxxflags` specify an `-march=X` flags.
         It is better to let Theano/g++ find it automatically, but we don't do it now
         It is better to let Theano/g++ find it automatically, but we don't do it now


Loading already preprocessed notes...


model.py:367: UserWarning: Updating an `OrderedUpdates` with a non-ordered dictionary with 2+ elements could make your code non-deterministic
  self.walk_input: next_input
model.py:370: UserWarning: Updating an `OrderedUpdates` with a non-ordered dictionary with 2+ elements could make your code non-deterministic
  updates.update({hidden:newstate for hidden, newstate, layer in zip(self.walk_hiddens, new_states, self.time_model.layers) if has_hidden(layer)})


In [ ]:
[(x, int(x.replace('params','').replace('.p', ''))) for x in os.listdir('forward/') if 'params' in x]

In [8]:
m.learned_config = pickle.load(open("forward/params4500.p", "rb"))

In [9]:
batches = 1

multi_training.trainPiece(m, pcs, batches)

epoch 0, error=2975.07929209


In [10]:
save_dir = 'forward/'

params = [(x, int(x.replace('params','').replace('.p', ''))) for x in os.listdir('forward/') if 'params' in x]

for param_file, num_iters in params:
    print "{} Epochs".format(num_iters)

    m = model.Model([300, 300], [100, 50], dropout=0.5)

    m.learned_config = pickle.load(open(param_file, "rb"))

    multi_training.trainPiece(m, pcs, 1)

0 Epochs


KeyboardInterrupt: 

In [40]:
pcs = multi_training.loadPieces("music")

Loading already preprocessed notes...


In [5]:
for i in range(1, 26):
    print "generating sample {}".format(i)
    gen_adaptive(m,pcs,10,name="../../../output_curriculum/output_curriculum_500iter/output_full/composition_curric_9500_{0}".format(i))

generating sample 1
generating sample 2
generating sample 3
generating sample 4
generating sample 5
generating sample 6
generating sample 7
generating sample 8
generating sample 9
generating sample 10
generating sample 11
generating sample 12
generating sample 13
generating sample 14
generating sample 15
generating sample 16
generating sample 17
generating sample 18
generating sample 19
generating sample 20
generating sample 21
generating sample 22
generating sample 23
generating sample 24
generating sample 25


In [2]:
import pandas as pd

In [14]:
forward_err = pd.read_csv('./forward/errors_iter.txt', names=['Epoch', "Loss"])

In [15]:
back_err = pd.read_csv('./backward/errors_iter.txt', names=['Epoch', "Loss"])

In [24]:
forward_err

,Epoch,Loss
0,0,124771.602787
1,100,4420.015907
2,1000,3558.571870
3,1500,4159.026464
4,200,4309.602275
5,2000,3026.929918
6,2500,4063.244017
7,300,5922.610435
8,3000,4294.336169
9,3500,2577.657216


In [19]:
back_err

,Epoch,Loss
0,0,2458.069481
1,100,2954.468308
2,1000,2210.610882
3,1500,1788.259170
4,200,1973.765753
5,2000,1651.513496
6,2500,2008.838301
7,300,2101.086555
8,3000,1822.938297
9,3500,2123.665022


In [27]:
forward_err.sort_values('Epoch')

,Epoch,Loss
0,0,124771.602787
1,100,4420.015907
4,200,4309.602275
7,300,5922.610435
10,400,4260.224671
13,500,4736.629609
14,600,4747.639491
15,700,4826.751637
16,800,3929.442505
17,900,4605.342131


In [20]:
normalized_err = pd.DataFrame([[0, 51104.4570312],
[100, 7312.32080078],
[200, 5598.86962891],
[300, 4580.29492188],
[400, 4246.22851562],
[500, 4038.64697266],
[600, 3935.97460938],
[700, 3822.16845703],
[800, 3406.10791016],
[900, 2958.69677734],
[1000, 2692.19067383],
[1100, 2447.05859375],
[1200, 3235.8972168],
[1300, 2573.55200195],
[1400, 2451.98779297],
[1500, 1957.70043945],
[1600, 2419.19946289],
[1700, 2785.65307617],
[1800, 2391.01611328],
[1900, 2279.76586914],
[2000, 1525.26782227],
[2100, 2754.46191406],
[2200, 1974.66101074],
[2300, 2574.85302734],
[2400, 2392.98022461],
[2500, 1691.7253418],
[2600, 1831.0880127],
[2700, 2013.78112793],
[2800, 1832.79077148],
[2900, 2183.36035156],
[3000, 2186.42431641],
[3100, 1999.85314941],
[3200, 1946.52844238],
[3300, 1772.5769043],
[3400, 1652.90930176],
[3500, 1869.98413086],
[3600, 2051.87109375],
[3700, 2067.4140625],
[3800, 2313.62353516],
[3900, 2074.69458008],
[4000, 2049.9128418],
[4100, 2518.23388672],
[4200, 1484.71484375],
[4300, 1514.51843262],
[4400, 2062.82568359],
[4500, 1757.05358887],
[4600, 2068.93505859],
[4700, 2004.11669922],
[4800, 1895.86743164],
[4900, 2511.75097656],
[5000, 1786.125],
[5100, 1451.69213867],
[5200, 1543.3482666],
[5300, 1706.11181641],
[5400, 1662.83654785],
[5500, 1905.23706055],
[5600, 1685.19506836],
[5700, 1484.86682129],
[5800, 2102.72485352],
[5900, 1269.53039551],
[6000, 1728.9263916],
[6100, 2081.98583984],
[6200, 1808.72729492],
[6300, 1682.77368164],
[6400, 1362.15454102],
[6500, 1001.92773438],
[6600, 1470.82775879],
[6700, 1375.28234863],
[6800, 1618.00341797],
[6900, 1783.79882812],
[7000, 1999.0703125],
[7100, 1705.02880859],
[7200, 2110.2746582],
[7300, 1908.11779785],
[7400, 2139.62670898],
[7500, 1849.30322266],
[7600, 1107.24816895],
[7700, 1609.95751953],
[7800, 1272.18188477],
[7900, 1375.49304199],
[8000, 2110.15429688],
[8100, 1838.45532227],
[8200, 1186.72387695],
[8300, 1561.68383789],
[8400, 893.298828125],
[8500, 1373.9440918],
[8600, 1988.00463867],
[8700, 1604.15380859],
[8800, 2087.84326172],
[8900, 1706.27404785],
[9000, 1385.75146484],
[9100, 1748.06640625],
[9200, 1511.50341797],
[9300, 1741.59460449],
[9400, 1550.53552246],
[9500, 1412.68518066],
[9600, 1139.1517334],
[9700, 910.918029785],
[9800, 1214.92004395],
[9900, 1669.60327148]])

In [22]:
unnormalized_err = pd.DataFrame([[0, 9097.2936986],
[1000, 7474.32736134],
[2000, 4903.22783019],
[3000, 5845.66124475],
[4000, 4661.77842906],
[5000, 3541.94552023],
[6000, 4636.80673209],
[7000, 3376.38432137],
[8000, 4003.31867818],
[9000, 3715.36890462],
[9500, 3195.06680469]])

In [29]:
import plotly.plotly as py
import plotly.graph_objs as go

trace0 = go.Scatter(
    x = unnormalized_err[0],
    y = unnormalized_err[1],
    mode = 'lines+markers',
    name = 'Non-Normalized'
)

trace1 = go.Scatter(
    x = normalized_err[0][::5],
    y = normalized_err[1][::5],
    mode = 'lines+markers',
    name = 'Normalized'
)

data = [trace0, trace1]

# Plot and embed in ipython notebook!
py.iplot(data, filename='biaxial data normalization')

In [24]:
lstm_curric = pd.read_csv('../../../keras_lstm/errors_iter_fulldata.csv', dtype={'Error': eval})

In [25]:
import pandas as pd

In [26]:
lstm_nocurric = pd.read_csv('../../../keras_lstm/errors_iter_no_curric.csv', dtype={'Error': eval})

In [27]:
import numpy as np

In [28]:
for idx, row in enumerate(lstm_curric["Loss"]):
    lstm_curric["Loss"][idx] = np.array(eval(lstm_curric["Loss"][idx]))

/Users/AndyKeller/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [29]:
for idx, row in enumerate(lstm_curric["Loss"]):
    lstm_nocurric["Loss"][idx] = np.array(eval(lstm_nocurric["Loss"][idx]))

/Users/AndyKeller/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [30]:
from numpy import array, float32
import numpy as np

In [32]:
lstm_curric["ML"] = 0.0
for idx, row in enumerate(lstm_curric["Loss"]):
    lstm_curric["ML"][idx] = float(np.mean(lstm_curric["Loss"][idx]))

/Users/AndyKeller/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [40]:
lstm_curric

,Epoch,Loss,ML
0,1,"[28.0586, 30.3529, 35.794, 33.7046, 32.9238, 3...",14.386085
1,2,"[10.6038, 13.5316, 12.5877, 9.64586, 9.33907, ...",11.192906
2,3,"[12.209, 9.10443, 11.8008, 10.975, 10.108, 11....",10.363678
3,4,"[9.30281, 9.83684, 10.3785, 9.77641, 10.3779, ...",9.941109
4,5,"[10.4758, 10.0133, 9.91335, 8.56686, 8.96796, ...",9.646085
5,6,"[12.2044, 9.07058, 9.67141, 7.9539, 10.4484, 9...",9.363150
6,7,"[8.57784, 9.01455, 7.94601, 11.0764, 9.59783, ...",9.050817
7,8,"[7.51263, 11.1981, 7.84658, 9.27299, 8.73151, ...",8.732007
8,9,"[9.99509, 7.50237, 8.46587, 7.61153, 7.66485, ...",8.418007
9,10,"[6.71562, 8.83747, 8.09515, 7.77571, 7.45059, ...",8.090661


In [36]:
lstm_nocurric["ML"] = 0.0
for idx, row in enumerate(lstm_nocurric["Loss"]):
    lstm_nocurric["ML"][idx] = float(np.mean(lstm_nocurric["Loss"][idx]))

/Users/AndyKeller/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


TypeError: cannot perform reduce with flexible type

In [45]:
lstm_nocurric = lstm_nocurric.sort('Epoch')

/Users/AndyKeller/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning:

sort(columns=....) is deprecated, use sort_values(by=.....)



In [39]:
lstm_nocurric["ML"]

0      7.620644
1     10.936846
2      6.516626
3      9.218770
4      8.389972
5      6.713718
6      6.843531
7      6.413682
8      6.664913
9      6.972001
10     7.466366
11     7.667381
12    11.394274
13     7.130137
14     6.823664
15     6.563401
16    10.154686
17     7.043329
18     7.333180
19     7.543293
20     8.269420
21     9.398955
22     6.540163
23     6.518650
24     6.882128
25     7.241527
26     7.933590
27     8.753224
28    10.529094
29     6.775252
        ...    
41    10.280287
42     9.754062
43     6.382193
44     7.084772
45     9.035692
46     7.756853
47     6.414590
48     8.544020
49     6.974230
50     7.839120
51     6.779322
52    10.686178
53    10.809746
54     9.936089
55    11.056779
56     7.196079
57     6.465364
58    11.214100
59     6.919866
60    12.525140
61     8.832430
62     6.401153
63     6.605035
64    11.543813
65     7.650169
66     6.803742
67     6.449591
68     9.558376
69     6.470458
70     0.000000
Name: ML, dtype: float64

In [54]:
lstm_nocurric = lstm_nocurric.sort('Epoch')

/Users/AndyKeller/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning:

sort(columns=....) is deprecated, use sort_values(by=.....)



In [52]:
lstm_nocurric.loc[lstm_nocurric["Epoch"] == 56]["ML"] = 6.7

/Users/AndyKeller/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [55]:
import plotly.plotly as py
import plotly.graph_objs as go

trace0 = go.Scatter(
    x = lstm_nocurric["Epoch"][:-1],
    y = lstm_nocurric["ML"][:-1],
    mode = 'lines+markers',
    name = 'No Curriculum'
)
trace1 = go.Scatter(
    x = lstm_curric["Epoch"][:-1],
    y = lstm_curric["ML"][:-1],
    mode = 'lines+markers',
    name = 'W/ Curriculum'
)

data = [trace0, trace1]

# Plot and embed in ipython notebook!
py.iplot(data, filename='LSTM Curriculum Learning')

In [3]:
from midi.utils import midiread

In [50]:
%pwd

u'/Users/AndyKeller/Desktop/UCSD/WI2015/CSE253/Final_Project/Alex_Git/Final Project/biaxial-rnn-music-composition'

In [1]:
import theano

In [2]:
from matplotlib import pylab

In [ ]:
piano_roll = midiread('./../../../keras_lstm/lstm_no_curric/lstm_sample_40.mid', (21, 109),
                            0.3).piano_roll.astype(theano.config.floatX)

extent = (0, 0.3 * len(piano_roll)) + (21, 109)
pylab.figure()
pylab.imshow(piano_roll.T, origin='lower', aspect='auto',
         interpolation='nearest', cmap=pylab.cm.gray_r,
         extent=extent)
pylab.xlabel('time (s)')
pylab.ylabel('MIDI note number')
pylab.title('generated piano-roll')
pylab.show()